https://www.metacritic.com/game 에서 크롤링 예정

In [7]:
import re
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup

헤더부분 유저에이전트,레퍼러 예제

In [8]:
hur={'User-Agent':'Mozilla','referer':'https://www.metacritic.com/game/playstation-5/elden-ring'}
url='https://www.metacritic.com/game/playstation-5/elden-ring'
req=requests.get(url,headers=hur)
bsoup=BeautifulSoup(req.content,'html.parser',from_encoding='utf-8') #beautifulsoup객체선언
q1=bsoup.find("div",{"class":"details main_details"})
temp=bsoup.find("span",{"class":"blurb blurb_expanded"}).text
print(temp)

A New World Created By Hidetaka Miyazaki And George R. R. Martin. ELDEN RING, developed by FromSoftware, Inc. and BANDAI NAMCO Entertainment Inc., is a fantasy action-RPG adventure set within a world created by Hidetaka Miyazaki creator of the influential DARK SOULS video game series; and George R.R. Martin author of The New York Times best-selling fantasy series, A Song of Ice and Fire. Danger and discovery lurk around every corner in FromSoftware's largest game to-date.  Hidetaka Miyazaki, President and Game Director of FromSoftware Inc. Known for directing critically-acclaimed games in beloved franchises including Armored Core, Dark Souls, and Sekiro: Shadows Die Twice.  George R.R. Martin is the #1 New York Times bestselling author of many novels, including the acclaimed series A Song of Ice and Fire - A Game of Thrones, A Clash of Kings, A Storm of Swords, A Feast For Crows, and A Dance with Dragons. As a writer-producer, he has worked on The Twilight Zone, Beauty and the Beast, a

이전까지 크롤링했던 진도부터 하기

In [9]:
f = open("proc.txt", 'r')
proc = int(f.readline())
f.close()

정해진 플랫폼에 따라 크롤링하는 함수

In [13]:
def crawl_page(url):
    ret={}
    chk=True
    print(url)
    while(chk):
        req=requests.get(url,headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36','referer':url}) 
        if(req==None): 
            print("reqNone")
            continue
        bsoup=BeautifulSoup(req.text,'lxml')
        time.sleep(1)#크롤링 속도 제한
        if(bsoup==None):
            print("bsoupNone")
            continue
        bmetascore=bsoup.find("span",{"itemprop":"ratingValue"})
        if(bmetascore==None):
            print(bsoup)
            print("bemtaNone")
            continue
        bsummary=bsoup.find("span",class_="blurb blurb_expanded")
        if(bsummary==None):
            print("bsumm")
            bsummary="None"
        else:
            if(bsummary.string==None):
                bsummary=bsoup.find("span",class_="blurb blurb_collapsed")
            else:
                bsummary=bsummary.string
        btitle=bsoup.find("div",{"class":"product_title"}).find("h1")
        if(btitle==None): 
            print("btitleNone")
            continue
        buserscore=bsoup.find("div",class_=re.compile("metascore_w user large game"))
        if(buserscore==None): 
            print("buserNone")
            continue
        bpub=bsoup.find("li",class_="summary_detail publisher").find("a")
        if(bpub==None):
            print("bpubNone")
            continue
        bdev=(bsoup.find("li",class_="summary_detail developer"))
        if(bdev==None):
            print("bdevNone")
            continue
        else:
            bdev=bdev.find("a")
        bgenres=(bsoup.find("li",class_="summary_detail product_genre"))
        if(bgenres==None):
            print("bgenreNone")
            continue
        bgenres=bgenres.find_all("span",class_="data")
        sgenres=""
        for x in range(len(bgenres)):
            sgenres=sgenres+str(bgenres[x].string)+"|"
        bgrating=bsoup.find("li",class_="summary_detail product_rating")
        if(bgrating==None):
            print("bgratingNone")
            bgrating="None"
        else:
            bgrating=bgrating.find("span",class_="data").string
        brelease=bsoup.find("li",class_="summary_detail release_data")
        brelease=brelease.find("span",class_="data")
        print(str(btitle.string)," ",str(brelease.string)," ",str(bmetascore.string)," ",str(buserscore.string)," ",str(bpub.string)[57:-53]," ",str(bdev.string)," ",sgenres," ",bgrating)
        ret={"title":str(btitle.string),"metascore":str(bmetascore.string),"userscore":str(buserscore.string),"publisher":str(bpub.string)[57:-53],"developer":str(bdev.string),"genres":sgenres,"rating":str(bgrating),"summary":str(bsummary),"release_date":str(brelease.string)}
        chk=False
    return ret
    
def get_page(url):
    req=requests.get(url,headers={'User-Agent':'Mozilla','referer':url})  
    bsoup=BeautifulSoup(req.content,'html.parser',from_encoding='utf-8')
    bsoup2=bsoup.find_all("td",{"class":"clamp-summary-wrap"})
    lpd=pd.DataFrame(columns={"title","release_date","publisher","developer","genres","rating","summary","metascore","userscore"})
    cnt=0
    for x in bsoup2:
        y=x.find("a")["href"]
        y=y[:-15]
        df_dic=pd.DataFrame([crawl_page("https://www.metacritic.com/"+y)])
        lpd = pd.concat([lpd, df_dic], ignore_index=True)
    return lpd
        
def crawl_direct(platform,df):
    url="https://www.metacritic.com/browse/games/release-date/available/"+platform+"/metascore?page="
    req=requests.get(url,headers={'User-Agent':'Mozilla','referer':url+'0'})
    bsoup=BeautifulSoup(req.content,'html.parser',from_encoding='utf-8')
    lpage_soup=bsoup.find("li",{"class":"last_page"})
    lpage2_soup=lpage_soup.find("a",{"class":"page_num"})
    lpagenum=lpage2_soup.text
    for x in range(proc,int(lpagenum)):       
        f = open("proc.txt", 'w')
        f.write(str(x))
        f.close()
        page=get_page(url+str(x))
        page.to_csv('ps4.csv',sep=',',na_rep='NaN',float_format = '%.2f',index = False)
crawl_direct('ps4',1)

https://www.metacritic.com//game/playstation-4/red-dead-redemption-2
Red Dead Redemption 2   Oct 26, 2018   97   8.6   Rockstar Games   Rockstar Games   Action Adventure|Open-World|   M
https://www.metacritic.com//game/playstation-4/grand-theft-auto-v
Grand Theft Auto V   Nov 18, 2014   97   8.3   Rockstar Games   Rockstar North   Action Adventure|Modern|Open-World|   M
https://www.metacritic.com//game/playstation-4/persona-5-royal
Persona 5 Royal   Mar 31, 2020   95   8.5   Atlus   Atlus   Role-Playing|Japanese-Style|   M
https://www.metacritic.com//game/playstation-4/the-last-of-us-remastered
The Last of Us Remastered   Jul 29, 2014   95   9.2   SCEA   Naughty Dog   Action Adventure|General|Modern|   M
https://www.metacritic.com//game/playstation-4/god-of-war
God of War   Apr 20, 2018   94   9.1   Sony Interactive Entertainment   SCE Santa Monica   Action Adventure|Linear|   M
https://www.metacritic.com//game/playstation-4/xcom-2-war-of-the-chosen
XCOM 2: War of the Chosen   Aug 29, 

Bastion   Apr  7, 2015   89   7.7   Supergiant Games   Supergiant Games   Role-Playing|Action RPG|   E10+
https://www.metacritic.com//game/playstation-4/apex-legends
Apex Legends   Feb  4, 2019   89   6.9   Electronic Arts   Respawn Entertainment   Action|Shooter|First-Person|Tactical|   T
https://www.metacritic.com//game/playstation-4/synth-riders
bsumm
Synth Riders   Aug 10, 2021   89   6.5   Kluge Interactive   Kluge Interactive   Action|Rhythm|Music|   T
https://www.metacritic.com//game/playstation-4/dreams
Dreams   Feb 14, 2020   89   8.7   SCEE   Media Molecule   Miscellaneous|General|Action Adventure|Sandbox|   T
https://www.metacritic.com//game/playstation-4/dragon-age-inquisition
Dragon Age: Inquisition   Nov 18, 2014   89   7.4   Electronic Arts   BioWare   General|Role-Playing|General|Action RPG|Western-Style|   M
https://www.metacritic.com//game/playstation-4/tony-hawks-pro-skater-1-+-2
Tony Hawk's Pro Skater 1 + 2   Sep  4, 2020   89   8.8   Activision   Vicarious Visions 

Okami HD   Dec 12, 2017   87   8.3   Capcom   Capcom   Action Adventure|General|   T
https://www.metacritic.com//game/playstation-4/towerfall-ascension
TowerFall Ascension   Mar 11, 2014   87   7.1   Matt Makes Games Inc.   Matt Thorson   Action|General|Platformer|2D|   E10+
https://www.metacritic.com//game/playstation-4/monster-boy-and-the-cursed-kingdom
Monster Boy and the Cursed Kingdom   Dec  4, 2018   87   8.1   FDG Entertainment   FDG Entertainment   Action|Action Adventure|General|Platformer|Metroidvania|   E10+
https://www.metacritic.com//game/playstation-4/fifa-14
FIFA 14   Nov 12, 2013   87   6.3   EA Sports   EA Canada   Sports|Traditional|Team|Soccer|Sim|Sim|   E
https://www.metacritic.com//game/playstation-4/doom-eternal
DOOM Eternal   Mar 20, 2020   87   8.9   Bethesda Softworks   id Software   Action|Shooter|First-Person|Arcade|   M
https://www.metacritic.com//game/playstation-4/pro-evolution-soccer-2016
Pro Evolution Soccer 2016   Sep 15, 2015   87   7.2   Konami   Kona

Guilty Gear Xrd: Rev 2   May 26, 2017   86   8.2   Arc System Works   Arc System Works   Action|Fighting|2D|   T
https://www.metacritic.com//game/playstation-4/olliolli2-welcome-to-olliwood
OlliOlli2: Welcome to Olliwood   Mar  3, 2015   86   6.5   roll7   roll7   Sports|Alternative|Individual|Skateboarding|Skate / Skateboard|   T
https://www.metacritic.com//game/playstation-4/resogun-heroes
Resogun: Heroes   Jun 24, 2014   86   7.3   SCEA   Housemarque   Action|Shooter|Scrolling|Shoot-'Em-Up|Horizontal|   E10+
https://www.metacritic.com//game/playstation-4/dirt-rally
DiRT Rally   Apr  5, 2016   85   7.9   Codemasters   Codemasters   Racing|Simulation|Automobile|   E
https://www.metacritic.com//game/playstation-4/crypt-of-the-necrodancer
Crypt of the NecroDancer   Feb  2, 2016   85   6.6   Brace Yourself Games   Brace Yourself Games   Action|Role-Playing|Rhythm|General|Action RPG|Music|   T
https://www.metacritic.com//game/playstation-4/13-sentinels-aegis-rim


KeyboardInterrupt: 